### Auto Approval Cron Job

1. Settings
    1. account ID : accID
    1. vehicle ID : vehID
    1. region
    1. Time range: timeRange
1. Get relevant alerts
    1. Fetch by {accID, timeRange} -- via API
    1. Filter alerts that have remarks/approval 
    1. Filter alert by vehID
1. For each alert get video from aws -- parallel thread
1. Eval for approval <bf>evalApproval(localVideoFile, alertType)</bf>
1. Log results:
    1. [alert_id, image, type, result, stat, run_ts, run_times]
    1. 12345678, <image>, seatbelt, approve, 867/1000, 12345678, (1, 1.3)
    1. 12345678, <image>, mobile, ignore, 67/300, 12345679, (1, 1.3)
1. Update alerts via API


In [1]:
import json, glob, os, sys
from datetime import datetime
import time
import utilsAWS as uAws;
import intanglesAPIs as iApis
import imageio
from importlib import reload
import numpy as np
import matplotlib.pyplot as plt
reload(uAws)
uAws.gVerb = False

from PIL import Image
import io
import base64

def getBase64Image(img):
    # Convert NumPy array to PIL Image
    pil_image = Image.fromarray(img)

    # Save the PIL Image to a BytesIO buffer in PNG format
    buffer = io.BytesIO()
    pil_image.save(buffer, format="jpeg")
    image_bytes = buffer.getvalue()

    # Base64 encode the image bytes
    base64_encoded_image = base64.b64encode(image_bytes).decode('utf-8')
    return base64_encoded_image

### 1. Settings

In [46]:
settings = {
    'accID': '621348950418460672', # Shrinath
    'accID': '1313274493921132544', # TMS US
    'tStart':  datetime(2025,7,15,8,0),
    'tEnd': datetime(2025,7,18,3),
    'region': 'India',
    'region': 'US',
    'vehID': None,
    'procDir': './AutoApprovalProc/',
}
settings['fetchAlertTypes'] = 'mobile_phone,seatbelt_violation'

vidType = {
    'mobile_phone': 'driver',
    'seatbelt_violation': 'driver',
}

 
if settings['region'].lower()=='us':
    urlUS = 'https://apis.intangles-aws-us-east-1.intangles.us'
    tokenUS = 'FCNlIrz1dHlHyd_QYxF9V8bmLLkxNm_IzA3vq73MXdhpqOcXPf1YGBNIlgnYvrRI'
    settings['urlFetchAlerts'] = urlUS
    settings['urlUpdateAlerts'] = 'https://apis.intangles-aws-us-east-1.intangles.us/alertlog/{alertID}/updateV2?&token={token}'
    settings['token'] = tokenUS
    uAws.initiate_s3_session(isUS=True)
else:
    settings['urlFetchAlerts'] = 'https://apis.intangles.com'
    settings['urlUpdateAlerts'] = 'https://apis.intangles.com/alertlog/{alertID}/updateV2?&token={token}'
    settings['token'] = 'JBRHjp1tPFdyZwvRGblGwi-hIv5OmYu-cr--qzRE9rSCY6F1M5vxQt5Y7Wn9g7ur' 
    uAws.initiate_s3_session(isUS=False)



In [47]:
    
info = {
    'start': datetime.now(),
    'settings': settings,
}

### 2. Fetch Alerts & Apply Filters

In [48]:
t1 = time.time()
dAlerts, info = iApis.getVTAlerts(st = settings['tStart'],
                            en = settings['tEnd'],
                            baseUrl = settings['urlFetchAlerts'],
                            utoken = settings['token'],
                            alertType= settings['fetchAlertTypes'],
                            accID = settings['accID'],
                           )
t2 = time.time()
info['t_fetchAlerts'] = t2-t1
info['n_alerts'] = len(dAlerts)
print("#Alerts=%d, took %d sec"%(len(dAlerts), t2-t1))



#Alerts=19, took 1 sec


In [49]:
from collections import Counter
Counter([x['type'] for x in dAlerts])

Counter({'seatbelt_violation': 17, 'mobile_phone': 2})

In [50]:
alertsToProcess = []
info['alertsToProcess'] = []
info['alertsToProcessMeta'] = []
info['alertsDropped'] = []
for ent in dAlerts:
    # filter to keep entries in vehID only (if given)
    vid = ent['vehicle_id']
    if settings['vehID'] is not None:
        if vid not in settings['vehID']:
            continue
            
    # check if alert has already been annotated
    if ent.get('alert_values',{}).get('approval_status',None) is not None:
        continue
        
    # check if video is available
    typ = ent['type']
    vidUrl = None
    for url in ent.get('alert_values',{}).get('artefacts',[]):
        if url.find(vidType[typ]+'.mp4')>=0:
            vidUrl = url
            
    if vidUrl is None:
        info['alertsDropped'].append((ent['id'], typ, 'no video file'))
    else:
        info['alertsToProcess'].append((ent['id'], typ, vidUrl))
        info['alertsToProcessMeta'].append(ent)
print('#dropped=',len(info['alertsDropped']))
print('#for process=',len(info['alertsToProcess']))

#dropped= 0
#for process= 1


In [51]:
Counter([x[1] for x in info['alertsToProcess']])

Counter({'seatbelt_violation': 1})

### Get Video from S3

In [52]:
session_id = datetime.now().strftime('%Y%m%d_%H%M')
videoDir = settings['procDir']+'/vids_%s/'%(session_id)
print(videoDir)
os.makedirs(videoDir, exist_ok=True)

files = []
for (aId, typ, vidUrl) in info['alertsToProcess']:
    files.append(vidUrl)
#     if len(files)>40:
#         break
    
t1 = time.time()
uAws.downloadFiles(files, outDirPath=videoDir)
t2 = time.time()
info['t_AWS'] = t2-t1

./AutoApprovalProc//vids_20250718_0311/


### Process each video via Dino

In [53]:
# initialize
import utilsAutoApprovalDino as uDino
reload(uDino)
uDino.init('AutoApprovalProc/assets/refData.json')

loading DINO model ...


Using cache found in C:\Users\Vikram.Melapudi/.cache\torch\hub\facebookresearch_dinov2_main
C:\Users\Vikram.Melapudi/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
C:\Users\Vikram.Melapudi/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
C:\Users\Vikram.Melapudi/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


loading reference dataset ...
mobile_phone (137, 768)
seatbelt_violation (235, 768)
DONE.


In [54]:
counts = {'Approve':0, 'Ignore':0, 'Verify': 0}
processedAlerts = {}

In [55]:
tempDir = os.path.join(settings['procDir'],'temp/')
os.makedirs(tempDir, exist_ok=True)
    
nVid = 0
for (aId, typ, vidUrl) in info['alertsToProcess']:
    if aId in processedAlerts:
        continue
    nVid += 1
    print('process %d/%d...'%(len(processedAlerts), len(info['alertsToProcess'])))
    t1 = time.time()
    localVidFile = uAws.s3ToLocalPath(vidUrl)
    vid = imageio.get_reader( os.path.join(videoDir,localVidFile) )
    N = 10 # vid.count_frames()
    img = vid.get_data(N//2)
    
    # handle full cabin image -> crop to corrrect half side
    if img.shape[1]>800:
        if settings['region'].lower()=='us':
            img = img[:,img.shape[1]//2:]
        else:
            img = img[:,:img.shape[1]//2]

    qobj = uDino.procImage(img, saveNpz=False)#, forceLoad=True)
    dists, indxs = qobj['knn'].kneighbors(uDino.feats[typ], 3)

    xyDists = [] # tuple of (x,y,D) for each reference patch
    for n in range(len(dists)):
        for m in range(3):
            x, y = uDino.index2xy(indxs[n,m], qobj)
            xyDists.append((x,y,dists[n,m]))
    xyDists = np.array(xyDists) 
    rules = uDino.refDataset.get(typ,{}).get('rules',{})
    # indx = np.where(xyDists[:,2]<rules['threshold'])[0]

    t2 = time.time()

    Nthresh = rules['passThreshold']
    Ntot = len(xyDists)
    Npass = np.sum(xyDists[:,2]<Nthresh)
    if Npass > Ntot//2+int(Ntot*rules['unsureFraction']):
        Nres = rules["passAction"]
    elif Npass < Ntot//2-int(Ntot*rules['unsureFraction']):
        Nres = rules["failAction"]
    else:
        Nres = 'Verify'

    print(os.path.basename(localVidFile), typ, Ntot, Npass, Nres, '(%d s)'%(t2-t1))
    counts[Nres] += 1

    rimg = uDino.cv2.resize(img, (300,200))
    
    processedAlerts[str(aId)] = {
        'id': str(aId),
        'img': getBase64Image(rimg),
        'type': str(typ),
        'result': Nres,
        'Npass': Npass,
        'Ntot': Ntot,
        'runS': (t2-t1),
        'date': datetime.now()
    }

    if False:
        plt.figure(figsize=(7,3))
        plt.subplot(121)
        plt.imshow(qobj['image'])
    #     plt.plot(xyDists[indx,0], xyDists[indx,1], 'rx')
        plt.subplot(122)
        plt.plot(sorted(xyDists[:,2]))
        plt.hlines(45,0,len(xyDists))#,'r-')
        plt.xlabel('feat-index')
        plt.ylabel('NN-dist')
        plt.tight_layout()
        plt.show()
print('')
print('DONE.')

process 0/1...
vod_1313375417322700800_2025-07-16_13-41-24-2025-07-16_13-41-25_driver.mp4 seatbelt_violation 705 290 Verify (5 s)

DONE.


### Write a Summary File

In [57]:
htmlFile = settings['procDir']+'/summary'+settings['region']+'_'+session_id+'.html'
print('HTML file=', htmlFile)
with open(htmlFile, 'w') as f:
    f.write('''<html><head>
    
    <style>
    *{
      font-family: Arial;
      font-size: 12px;
      }
    th, td {
      padding-left: 10px;
      padding-right: 10px;
    }
    .approve {
        background-color:#DDFFDD;
        padding:5px;
        border-radius:8px;
    }
    .ignore {
        background-color:#FFDDDD;
        padding:5px;
        border-radius:8px;
    }
    .verify {
        background-color:#FFF955;
        padding:5px;
        border-radius:8px;
    }
    </style>
    <script>
    var settingsJSON = '%s';
    var settingsRegion = '%s';
    function clicked(el) {
      var els = document.getElementsByClassName(el.id);
      console.info(el.id+','+els.length);
      var dval = 'none';
      if(el.checked) dval = '';
      for(var n=0; n<els.length; n++) {
        if(els[n].tagName=='LABEL') continue;
        els[n].style.display = dval;
      }
    }
    
    function initialize() {
      var el;
      var lst = ['approve','ignore','verify'];
      for(var n=0; n<3; n++) {
        el = document.getElementById("l"+lst[n]);
        var vels = document.getElementsByClassName(lst[n]);
        el.innerHTML = lst[n] + "(" + (vels.length-1) + ")";
        el.click();
        el.click();
      }
      el.click();
      
      // add edit option
      var vels = document.getElementsByTagName("tr");
      var sel = "<option value='same' default>same</option>";
      var selA= "<option value='approve'>approve</option>";
      var selI= "<option value='ignore'>ignore</option>";
      for(var m=1; m<vels.length; m++) {
        var idIndex = 0 + (vels[m].childNodes.length>7);
        el = vels[m].childNodes[3 + idIndex];
        el.innerHTML = el.innerHTML;
        var selector = sel+selA+selI
        if(el.innerHTML.toLowerCase().search("approve")>=0) selector = sel+selI;
        if(el.innerHTML.toLowerCase().search("ignore")>=0) selector = sel+selA;
        var alertId = vels[m].childNodes[idIndex].innerHTML;
        el.innerHTML += " <br><select style=padding:3px;margin-top:5px; id="+alertId+" typ="+vels[m].childNodes[2].innerHTML+">"+selector+"</select>";
      }
    }
    
    function makeXHRRequest(method, url, data = null) {
      return new Promise((resolve, reject) => {
        const xhr = new XMLHttpRequest();
        xhr.open(method, url, true); // true for asynchronous

        xhr.onload = function() {
          if (xhr.status >= 200 && xhr.status < 300) {
            resolve(xhr.responseText); // Resolve with the response text
          } else {
            reject(new Error(`Request failed with status ${xhr.status}`)); // Reject on error
          }
        };

        xhr.onerror = function() {
          reject(new Error('Network error or request failed')); // Handle network errors
        };

        if (data) {
          xhr.setRequestHeader('Content-Type', 'application/json'); // Example for JSON data
          xhr.send(JSON.stringify(data));
        } else {
          xhr.send();
        }
      });
    }
    
    function applyEdits() {
      var editList = [];
      var els = document.getElementsByTagName("select");
      for(var n=0; n<els.length; n++) {
        if(els[n].value!='same') {
          typ = els[n].getAttribute("typ");
          editList.push([els[n].id,typ,els[n].value]);
        }
      }
      alert("# entries to be updated:"+editList.length);
      
      var token = prompt("enter user token:");
      if(token.length==0) {
          alert("token length is 0");
      } else {
          var baseUrl = "https://apis.intangles.com";
          if (settingsRegion.toLowerCase().search("us")>=0) baseUrl = "https://apis.intangles-aws-us-east-1.intangles.us";
          
          var origUrl = baseUrl + "/alertlog/{alertID}/updateV2?token={token}";
          for(var n=0; n<editList.length; n++) {
              var url = origUrl.replaceAll('{alertID}', editList[n][0]);
              url = url.replaceAll('{token}', token);
              console.info(url);
              
              var status = 'decline';
              if (editList[n][2]=='approve') status = 'approve'              
              var jdata = {"approval_status": status, "remarks":"auto approval correction 1"};           
              var fdata = {"user_show": status=="approve", "alert_values": jdata}; //JSON.stringify(jdata)};
              console.info(editList[n][0], fdata);
              
              makeXHRRequest('POST', url, fdata)
                  .then(response => {
                    console.info('Data received:', response);
                  })
                  .catch(error => {
                    console.info('Error:', error);
                  });
          }
      }
    }
    </script>
    </head>
    <body style="text-align:center;" onload="initialize()">
    <h2 style="font-size:24px;background-color:#BBBBFF; padding:5px;">Auto Approval Review</h2>
    <div style="margin:10px"> accID=%s, vIDs=%s, time=[%s, %s]
    </div>
    <div style="margin:10px">
    <input type="checkbox" id="approve" onclick="clicked(this)"><label for="approve" id="lapprove" class="approve">approve</label>
    <input type="checkbox" id="ignore" onclick="clicked(this)"><label for="ignore" id="lignore" class="ignore">ignore</label>
    <input type="checkbox" id="verify" onclick="clicked(this)"><label for="verify" id="lverify" class="verify">verify</label>
    <button onclick="applyEdits()" style="padding:5px;margin-left:20px;">Apply Edits</button>
    </div>
    <table style="margin:5px; margin-left:auto; margin-right:auto;">
    '''%(json.dumps(settings,default=str), 
         settings['region'],
         settings['accID'], 
         str(settings['vehID']), 
         str(settings['tStart']), str(settings['tEnd'])) )
    
    header = ['Account ID', 'Alert ID','Image','Type','Result','Score','Timing','Date']
    f.write('<tr>')
    for head in header:
        f.write('<th>%s</th>'%head)
    f.write('</tr>\n')
    
    for idd in processedAlerts:
        lst = processedAlerts[idd]
        dtFmt = '%Y/%m/%d %H:%M'
        alertDate = datetime.fromtimestamp(int(lst['id'].split('-')[-1])/1000)
        f.write('<tr class="%s">'%lst['result'])
        f.write('<td>'+settings['accID']+'</td>')
        f.write('<td>'+lst['id']+'</td>')
        f.write('<td><img src="data:image/jpeg;base64,%s"></td>'%lst['img'])
        f.write('<td>'+lst['type']+'</td>')
        f.write('<td>%s</td>'%lst['result'])
        f.write('<td>%d/%d</td>'%(lst['Npass'], lst['Ntot']))
        f.write('<td>%1.2f</td>'%lst['runS'])
        f.write('<td>%s<br>(%s)</td>'%(alertDate.strftime(dtFmt), lst['date'].strftime(dtFmt)))
        f.write('</tr>\n')
    
    f.write('''</table>
    </body></html>
    '''
    )
    
      

HTML file= ./AutoApprovalProc//summaryUS_20250718_0311.html


In [31]:
from IPython.display import HTML

# HTML(filename=htmlFile)

### Update alerts

In [32]:
import requests

userShow = {
    'mobile_phone': {
        'Approve': (True, json.dumps({"approval_status":"approve", "remarks":"auto approval"})),
        'Ignore': (False, json.dumps({"approval_status":"decline", "remarks":"auto approval"})),
    },
    'seatbelt_violation': {
        'Approve': (True, json.dumps({"approval_status":"approve", "remarks":"auto approval"})),
        'Ignore': (False, json.dumps({"approval_status":"decline", "remarks":"auto approval"})),
    },
}


# ignoreData = {"user_show":False, 
#               "alert_values": json.dumps({"approval_status":"decline", "remarks":"Detection Issue: No yawn/No Eye Close"})
#              }

if True:
    alertUpdateURL = settings['urlUpdateAlerts']
    for aId in processedAlerts:
        ent = processedAlerts[aId]
        if ent['result']=='Verify':
            continue
        if ent['result']=='Approve':
            continue

        accId = settings['accID']
        token = settings['token']
        flag, values = userShow.get(ent['type']).get(ent['result'])

        ignoreData = {'user_show': flag, 'alert_values': values}
        head = ''
        if ent['result'].lower()=='approve':
            head = '***'
        print(head, aId, typ, ent['result'], flag)# , values)
        r = requests.post(alertUpdateURL.format(alertID=aId,token=token), json=ignoreData)
        rjd = r.json()


 1380918636398510080-seatbelt_violation-1752541729100 seatbelt_violation Ignore False
 1313375417322700800-seatbelt_violation-1752546130005 seatbelt_violation Ignore False


In [3]:
settings['urlUpdateAlerts']

'https://apis.intangles.com/alertlog/{alertID}/updateV2?&acc_id={accID}&token={token}'

In [36]:
import requests

aId = '1220369221305761792-mobile_phone-1752685303552'
token = settings['token']
alertUpdateURL = settings['urlUpdateAlerts']

values = json.dumps({"approval_status":"decline", "remarks":"auto approval correction"})
ignoreData = {'user_show': False, 'alert_values': values}
print(aId)

r = requests.post(alertUpdateURL.format(alertID=aId, token=token), json=ignoreData)
rjd = r.json()

1220369221305761792-mobile_phone-1752685303552


In [38]:
token

'JBRHjp1tPFdyZwvRGblGwi-hIv5OmYu-cr--qzRE9rSCY6F1M5vxQt5Y7Wn9g7ur'

In [6]:
alertIdsToIgnore='''
1367818721430601728-mobile_phone-1752497944948
1422927266731524096-mobile_phone-1752500868420
1422927266731524096-mobile_phone-1752500975565
621824374126735360-mobile_phone-1752459954020
621824374126735360-mobile_phone-1752460202149
1347204086562291712-mobile_phone-1752462349390'''

# alertIdsToIgnore = '''
# 1296113529593528320-seatbelt_violation-1752510042234
# 1296113529593528320-seatbelt_violation-1752515961831'''

import requests
alertUpdateURL = settings['urlUpdateAlerts']
for aId in alertIdsToIgnore.split('\n'):
    if len(aId.strip())<=0:
        continue
    
    accId = aId.split('-')[0]
    token = settings['token']
    values = json.dumps({"approval_status":"decline", "user_remarks":"auto approval correction"})
    ignoreData = {'user_show': False, 'alert_values': values}
    head = ''
    print(head, aId)
    r = requests.post(alertUpdateURL.format(alertID=aId,token=token), json=ignoreData)
    rjd = r.json()


 1296113529593528320-seatbelt_violation-1752510042234
 1296113529593528320-seatbelt_violation-1752515961831


In [7]:
alertIdsToAccept='''
1332391193698369536-mobile_phone-1752497932939
1359906014471651328-mobile_phone-1752499356739
711353357381402624-seatbelt_violation-1752423631550
1334833384924381184-mobile_phone-1752425481620'''


import requests
alertUpdateURL = settings['urlUpdateAlerts']
for aId in alertIdsToAccept.split('\n'):
    if len(aId.strip())<=0:
        continue
    
    accId = aId.split('-')[0]
    token = settings['token']
    values = json.dumps({"approval_status":"approve", "user_remarks":"auto approval correction"})
    ignoreData = {'user_show': True, 'alert_values': values}
    head = ''
    print(head, aId)
    r = requests.post(alertUpdateURL.format(alertID=aId,token=token), ignoreData)
    rjd = r.json()

 1332391193698369536-mobile_phone-1752497932939
 1359906014471651328-mobile_phone-1752499356739
 711353357381402624-seatbelt_violation-1752423631550
 1334833384924381184-mobile_phone-1752425481620
